# Simple Animations with `matplotlib`
You can create simple applications and even videos with `matplotlib`.

## A (very) simple imshow-animation

An animation consists of a sequence of images that are shown one after the other. We can create simple animations manually.

In [ ]:
# for animations, it seems we cannot use the inline matplotlib
# feature easily. It only shows me one single plot. If anyone
# knows a solution, please let me know!
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

resolution = 80
pixels = np.zeros((resolution, resolution))

# Idea: we create an initial plot and modify its contents
# in certain intervals of time. We need to use the
# 'object-oriented matplotlib approach for this.

# store the initial plot in a variable
p = plt.imshow(pixels)
p.set_cmap('Spectral') # color map for the plot.
plt.clim(0.0, 1.0)     # limits for the colour map
plt.colorbar()         # show colorbar

# draw a diagonal line 'in a movie' (resolution-1, resolution-1)
for i in range(resolution):
    pixels[i,i] = i / resolution # modifiy pixel-value
    p.set_data(pixels)           # renew the plot
    plt.pause(0.05)              # set plotting speed (interval between images)
    
plt.show()    

<a id='particles_in_box'></a>
## Particles in a box - a scatter-plot animation

Here a more advanced example with an animated scatter plot. The techniques in this example could be useful for this weeks homework assignment ([Dance of the moons]()).

In [5]:
# script to show the movements of three (point-like) particles within a box
# only interaction with the box boundaries, not netween the particles
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

# create figure manually; necessary for an animated figure
fig = plt.figure(figsize=(7, 7))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], frameon=True)

# The box has a side-length of 100
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)

# choose random start positions, velocities and colors for the particles;
# data structures are chosen to be convenient later-on
n_part = 10
pos = np.random.uniform(0, 100, (n_part, 2))
v_x = np.random.uniform(-5, 5, n_part)
v_y = np.random.uniform(-5, 5, n_part)
color_strings = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
col = np.random.choice(color_strings, n_part)

# create initial scatter plot
scat = ax.scatter(pos[:,0], pos[:,1], s=200, c=col)

steps = 100
# now do the animation:
for i in range(steps + 1):
    # change the speed if a particle touches the box
    # boundaries
    v_x[(pos[:,0] < 0) | (pos[:,0] > 100)] *= -1
    v_y[(pos[:,1] < 0) | (pos[:,1] > 100)] *= -1

    pos[:,0] += v_x
    pos[:,1] += v_y

    scat.set_offsets(pos)
    ax.set_title('step: {0} / {1}'.format(i, steps))
    plt.pause(0.05)

plt.show()

## The Double Pendulum - Differential equations and animations
Finally an example that shows how you can create videos. The basic techniques are the same as above. You need to write a function `plt_update` which updates the plot at each time step. The movie and the video are created with the function `FuncAnimation`-function within the `matplotlib.animation`-module.

**Notes:**
(1) The creation and display of animations created with the `FuncAnimation`-framwork depend quite heavily on your environment and external programs (e.g. `ffmpeg` or `avonv` to create videos). It is probable that you need to play around with settings and additional installations to get it working - especially on *Microsoft-Windows*. The *manual* techniques above are better behaved in that sense and you should use them if you do not need video-files; (2) Make sure that you have `ffmepg` or `avconv` installed on your system. Windows users can obtain binary versions of `ffmpeg` [here](https://ffmpeg.zeranoe.com/builds/).

<img src="figs/Double-compound-pendulum-dimensioned.svg" style="height: 200px;" style="width: 200px">

The ODEs covering the double-pendulum are just copied from [Wikipedia](https://en.wikipedia.org/wiki/Double_pendulum).

${\dot \theta_1} = \frac{6}{m\ell^2} \frac{ 2 p_{\theta_1} - 3 \cos(\theta_1-\theta_2) p_{\theta_2}}{16 - 9 \cos^2(\theta_1-\theta_2)}$

${\dot \theta_2} = \frac{6}{m\ell^2} \frac{ 8 p_{\theta_2} - 3 \cos(\theta_1-\theta_2) p_{\theta_1}}{16 - 9 \cos^2(\theta_1-\theta_2)}.$

${\dot p_{\theta_1}} = -\frac{1}{2} m \ell^2 \left [ {\dot \theta_1} {\dot \theta_2} \sin (\theta_1-\theta_2) + 3 \frac{g}{\ell} \sin \theta_1 \right ]$

${\dot p_{\theta_2}} = -\frac{1}{2} m \ell^2 \left [ -{\dot \theta_1} {\dot \theta_2} \sin (\theta_1-\theta_2) +  \frac{g}{\ell} \sin \theta_2 \right]$

I transfered the equations to the notation that we introduced last lecture.
$y = [\theta_1, \theta_2, p_{\theta_1}, p_{\theta_2}]$

${\dot y_1} = \frac{6}{m\ell^2} \frac{ 2 y_3 - 3 \cos(y_1-y_2) y_4}{16 - 9 \cos^2(y_1-y_2)}$

${\dot y_2} = \frac{6}{m\ell^2} \frac{ 8 y_4 - 3 \cos(y_1-y_2) y_3}{16 - 9 \cos^2(y_1-y_2)}$

${\dot y_3} = -\frac{1}{2} m \ell^2 \left [ {\dot y_1} {\dot y_2} \sin (y_1-y_2) + 3 \frac{g}{\ell} \sin y_1 \right ]$

${\dot y_4} = -\frac{1}{2} m \ell^2 \left [ -{\dot y_1} {\dot y_2} \sin (y_1-y_2) +  \frac{g}{\ell} \sin y_2 \right]$

In [1]:
import numpy as np
import scipy.integrate as si
import matplotlib.pyplot as plt
import matplotlib.animation as ma

# note (specifically for Windows users). If the ffmpeg/avconv
# program is not in your PATH, you can specify its location
# with a line such as the following:
#plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

def f(y, t, g, L, m):
    """
    The right-hand side of the double pendulum ODE
    """
    y1, y2, y3, y4 = y[0], y[1], y[2], y[3]
    
    dy1dt = 6.0 / (m * L**2) * \
            (2 * y3 - 3 * np.cos(y1 - y2) * y4) / (16 - 9 * np.cos(y1 - y2)**2)
    dy2dt = 6.0 / (m * L**2) * \
            (8 * y4 - 3 * np.cos(y1 - y2) * y3) / (16 - 9 * np.cos(y1 - y2)**2)
    dy3dt = -0.5 * m * L**2 * \
            ( dy1dt * dy2dt * np.sin(y1 - y2) + 3 * (g / L) * np.sin(y1))
    dy4dt = -0.5 * m * L**2 * \
            (-dy1dt * dy2dt * np.sin(y1 - y2) + (g / L) * np.sin(y2))
    
    return [ dy1dt, dy2dt, dy3dt, dy4dt ]

# physical constants involved in the formulae
g = 9.82
L = 0.5
m = 0.1

# choose an initial state; the first two coordinates represent
# the angles of the pendulum, the other two the angular
# velocities
y0 = [ 0.2 * np.pi / 4, 0.2 * np.pi / 2, 0.0, 0.0]

# time coodinate to solve the ODE for: from 0 to 10 seconds
t = np.linspace(0, 10, 250)

# solve the ODE problem
y = si.odeint(f, y0, t, args=(g, L, m))

# The following code part creates a Video of the double pendulum
# motion; see the matplotlib-animation documentation for more
# information on this topic!
# The output is a file 'animation.mp4' in the cwd.
fig, ax = plt.subplots(figsize=(7,7))

ax.set_ylim([-1.5, 1.5])
ax.set_xlim([1, -1])

pendulum1, = ax.plot([], [], color="red", lw=4)
pendulum2, = ax.plot([], [], color="blue", lw=4)

# Note: It nearly cannot be avoided to use 'global variables'
# for the two following functions!!
def init():
    pendulum1.set_data([], [])
    pendulum2.set_data([], [])
    
    # we need to return the figure elemnts that are
    # initialised
    return pendulum1, pendulum2

def update(n): 
    # n = frame counter
    # calculate the positions of the pendulums
    x1 = + L * np.sin(y[n, 0])
    y1 = - L * np.cos(y[n, 0])
    x2 = x1 + L * np.sin(y[n, 1])
    y2 = y1 - L * np.cos(y[n, 1])
    
    # update the line data
    pendulum1.set_data([0 ,x1], [0 ,y1])
    pendulum2.set_data([x1,x2], [y1,y2])
    
    # We need to return the figure-elements that
    # are modified by the update function
    return pendulum1, pendulum2

anim = ma.FuncAnimation(fig, update, init_func=init, frames=len(t), blit=True)

# The following command needs the program 'avconv' installed!
anim.save('animation.mp4', fps=20, writer="avconv", codec="libx264")
# The following variant of the anim.save command needs 'ffmpeg' installed
# (It is an alternative to avconv!)
#anim.save('animation.mp4', fps=20, writer="ffmpeg", codec="mpeg4")
plt.close(fig)

In [ ]:
# The following code displays the Video created
# for the motion of the double pendulum
# It may be that this does not work withon your browser
# (no support of mpeg4 files etc.). But you should be
# able to display the video with programs such as vlc
import io
import base64
from IPython.display import HTML

video = io.open('animation.mp4', 'r+b').read()
encoded = base64.b64encode(video)

HTML(data='''<video alt="test" controls>
     <source src="data:video/mp4;base64,{0}" type="video/mp4" />
     </video>'''.format(encoded.decode('ascii')))